# Model Deployment
Models can be deployed locally, to an Azure Container Instance, or to Azure Kubernetes Service

## Load Resources

In [40]:
# Load the workspace information from config.json using the Azure ML SDK
from azureml.core import Workspace

ws = Workspace.from_config()
ws.name

'DataScience'

In [41]:
from azureml.core.model import Model

# Grab the latest version of the registered model from Azure
model = Model(ws, name='NHL-Penalties-AutoML')
print('Loaded Model ' + model.name + ' v' + str(model.version))

Loaded Model NHL-Penalties-AutoML v5


## Configure Endpoint

In [42]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

# Load the environment from the YAML file downloaded from the best run
env = Environment.from_conda_specification("AutoML-env", "automl-output/outputs/conda_env_v_1_0_0.yml")

# Create an inference config pointing at the files we downloaded. This configuration tells Azure how to handle requests
inference_config = InferenceConfig(environment=env, 
                                   source_directory='./automl-output/outputs', 
                                   entry_script='./scoring_file_v_2_0_0.py')


In [47]:
from azureml.core.webservice import AciWebservice

# Set up keys for communicating with the service
key1 = 'HelloAutoML'
key2 = 'AnotherKey'

# The deployment configuration configures how the endpoint is hosted
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    enable_app_insights=True,
    auth_enabled=False) 

## Deploy Endpoint

In [48]:
# Deploy the model
service = Model.deploy(ws, "penalty-predictor", [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output = True)

# Grab our scoring endpoint for testing
print('Endpoint active at ' + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-18 22:02:22-04:00 Creating Container Registry if not exists.
2022-06-18 22:02:22-04:00 Registering the environment.
2022-06-18 22:02:23-04:00 Use the existing image.
2022-06-18 22:02:24-04:00 Generating deployment configuration.
2022-06-18 22:02:25-04:00 Submitting deployment to compute.
2022-06-18 22:02:28-04:00 Checking the status of deployment penalty-predictor..
2022-06-18 22:04:19-04:00 Checking the status of inference endpoint penalty-predictor.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Endpoint active at http://5e070320-2fcf-4b74-b0c5-00c728febae3.northcentralus.azurecontainer.io/score


In [49]:
import requests
import json

# Test the endpoint by having the Blue Jackets host the Rangers in the regular season
data = {
  "Inputs": {
    "data": [{"type": "R", "homeTeam": "Blue Jackets", "awayTeam": "Rangers"}]
  },
  "GlobalParameters": 1.0
}

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(service.scoring_uri, input_data, headers=headers)
results = resp.json()['Results']
results

[25.08131977502827]